<a href="https://colab.research.google.com/github/Sela80/s/blob/main/donn%C3%A9es_sur_le_diab%C3%A8te.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets --upgrade --quiet

In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, RocCurveDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import seaborn as sns
import joblib

In [ ]:
# Importation d'une Base de données Depuis Kaggle
import opendatasets as od
od.download('https://www.kaggle.com/datasets/akshaydattatraykhare/diabetes-dataset')

In [ ]:
df=pd.read_csv('/content/diabetes-dataset/diabetes.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.columns

In [ ]:
df.describe().transpose()

#Prétraitement

In [ ]:
x = df.drop('Outcome', axis=1)
y = df['Outcome']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
#Calcule le nombre maximale de variable(max_features) à la racine carrée
max_features = int(np.sqrt(x.shape[1]))
#Créer la forêt aléatoire avec les hyperparamètres fixés
random_Forest = RandomForestClassifier(n_estimators=100, max_features=max_features, random_state=42, oob_score = True)
#Entraîner le modèle sur les données d'entraînement
random_Forest.fit(x_train, y_train)

#Predire sur les bases de Train et Test

In [ ]:
# Predire les classes sur l'ensemble  de Train et Test
y_train_pred = random_Forest.predict(x_train)
y_test_pred = random_Forest.predict(x_test)

In [ ]:
#Calcule les mesures de performance
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, RocCurveDisplay
from sklearn.metrics import recall_score
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
train_auc = roc_auc_score(y_train, y_train_pred)
test_auc = roc_auc_score(y_test, y_test_pred)
train_recall = recall_score(y_train, y_train_pred)
test_recall = recall_score(y_test, y_test_pred)

#Créer le tableau d'évaluation de performance

performance_Table = pd.DataFrame({
    'Metrique' : ['Accuracy', 'AUC', 'Recall'],
    'Ensembe_d\'Entrainement' : [train_accuracy, train_auc, train_recall],
    'Ensemble_Test' : [test_accuracy, test_auc, test_recall]
})

# Afficher le tableau d'évaluation
print(performance_Table)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
#Obtenir le score de probability pour les classes positive

y_train_proba = random_Forest.predict_proba(x_train)[:, 1]
y_test_proba = random_Forest.predict_proba(x_test)[:, 1]

#Calcule des faux positifs(FPR) et les taux de vrais positifs (TPR)
fpr_train, tpr_train, _ = roc_curve(y_train, y_train_proba)
fpr_test, tpr_test, _ = roc_curve(y_test, y_test_proba)

#Comparaison de Train_socore et test_score
train_score = random_Forest.score(x_train, y_train)
test_score = random_Forest.score(x_test, y_test)
print(f"Train_score: {train_score:.2f}")
print(f"Test_score: {test_score:.2f}")

In [ ]:
# Trace de la courbe ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr_train, tpr_train, label='Ensembe_d\'Entrainement')
plt.plot(fpr_test, tpr_test, label='Ensemble_Test')
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Courbe ROC')
plt.legend()
plt.show()

# Importance des variables

In [ ]:
# Obtenir l'importance des variables
importance_df = pd.DataFrame({
    'Variable': x.columns,
    'Importance': random_Forest.feature_importances_
})

importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Afficher les 10 variables les plus importantes
print("Les 10 variables les plus importantes sont :")
print(importance_df.head(10))

In [ ]:
#Trace l'importance des variables

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Variable', data=importance_df.head(10))
plt.title('Importance des variables')
plt.show()

In [ ]:
joblib.dump(random_Forest, 'RandomForestRegressor.pkl')

In [ ]:
from google.colab import files
files.download('RandomForestRegressor.pkl')